<h1>Numerická práce s dataframy, výpočet hodnot a statistik</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os

In [ ]:
df_csob = pd.read_excel(os.path.join("dataset", "csob.xls"))
df_csob

In [ ]:
df_sporitelna = pd.read_csv(os.path.join("dataset", "sporitelna.csv"),
                            delimiter=';',
                            decimal=",")
df_sporitelna

<h5>Nahrazení chybějících hodnot</h5>

V obou tabulkách můžeme vidět velké množství hodnot NaN (not a number), které reprezentují prázdné buňky tabulky. <br>

Knihovna pandas nabízí metodu, pomocí které lze tyto chybějící hodnoty nahradit libovolnou hodnotou (číselnou či textem): <br>

`pd.DataFrame.fillna()`
<https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html>


In [ ]:
# nahrazení NaN slovy "Neni dostupne"

df_sporitelna.fillna("Neni dostupne", inplace=True)
df_sporitelna

<h5>Práce s časovými daty</h5>

Modul pandas obsahuje vlastní Datetime objekt (různý od standardního Pythonového datetime) a řadu metod, pomocí kterých lze s datetime objekty pracovat a upravovat je

In [ ]:
# naše dataframy obsahují sloupec s daty korelující k pohybům na účtech, ve většině případů dokáže modul Pandas časová data najít a správně převést na objekt typu Datetime

df_csob["date of posting"]

In [ ]:
df_csob["date of posting"][13]

In [ ]:
# z objektu typu Pandas timestamp můžeme získavat jednotlivé informace jako den, měsíc a rok, případně hodiny, minuty a sekundy pokud jsou specifikovány

print(df_csob["date of posting"][13].day_name())
print(df_csob["date of posting"][13].day)
print(df_csob["date of posting"][13].month_name())
print(df_csob["date of posting"][13].month)
print(df_csob["date of posting"][13].year)

In [ ]:
# pomocí timestampů můžeme v dataframu chytře vyhledávat - klíčové slovo nemusí být stejné jako zobrazené datum, stačí aby mělo stejný (časový) význam

df_csob[(df_csob["date of posting"] >= '1 Jul 2022'
         ) & (df_csob["date of posting"] < '15.07.2022')]

In [9]:
# Pandas dokáže k jednotlivým timestampům uchovávat informace o časovém pásmu

localized_stamp = pd.Timestamp(df_csob["date of posting"][13], tz="CET")

In [ ]:
localized_stamp.tz

In [ ]:
# Pandas Datetime je možné využít jako speciální typ indexu - DatetimeIndex, kerý nám umožní s dataframem jednoduše manipulovat pomocí dat a časů

df_csob.set_index(pd.DatetimeIndex(df_csob["date of posting"]),inplace=True, drop=True)
df_csob

In [ ]:
# filtrování dataframu na základě časového intervalu jako indexů

df_csob.loc["28/6/2022":"7/7/2022"]

In [ ]:
# filtrace dataframu na základě typu timestampu - např. poslední týden (week)

df_csob.sort_index().last("W")

<h5>Výpočetní operace se sloupci</h5>

Objekt typu Pandas dataframe umožňuje výpočetní operace na sloupcích po jednotlivých hodnotách bez nutnosti cyklů či iterativního postupu

In [ ]:
# V našem výpisu z účtu bychom rádi viděli, jak by vypadaly naše výdaje v amerických dolarech, převedeme tedy částky dělením 

dollar_value = 25.40 #23.9.2022
df_sporitelna_usd = df_sporitelna.copy(deep=True)
df_sporitelna_usd["Castka"] = round(df_sporitelna_usd["Castka"]/dollar_value,2) # zaokrouhlíme kvůli přehlednosti
df_sporitelna_usd

In [15]:
# Nyní musíme však přeznačit měnu částky, můžeme tedy hodnoty celého sloupce převést na americký dolar a to několika způsoby
# Pro zkoušku vytvořte novou kopii df_sporitelna_usd spuštěním předchozí buňky a pak příslušného způsobu

dollar_code = "USD"

In [ ]:
# způsob 1: nejjednodušší přiřazením hodnoty sloupci

df_sporitelna_usd["Mena"] = dollar_code
df_sporitelna_usd

In [ ]:
# způsob 2: pomocí metody df.loc()

df_sporitelna_usd.loc[:, "Mena"] = dollar_code
df_sporitelna_usd

In [ ]:
# způsob 3: pomocí metody df.assign()

df_sporitelna_usd.assign(Mena=dollar_code)

In [ ]:
# podobně můžeme provádět velké množství výpočetních operací se sloupci, které model pandas provede pro každý prvek zvlášť
# příklad na novém dataframu ze slovníku s mocninami 2

powers_of_two = {"N":[x for x in range(10)],"Power of 2":[np.power(2,y) for y in range(10)]}
powers_of_two = pd.DataFrame(powers_of_two)
powers_of_two.set_index("N",inplace=True)
powers_of_two

In [ ]:
# výpočty provádíme pomocí knihovny numpy, se kterou je modul Pandas kompatibilní (například nativní modul Pythonu math nemusí správně pochopit hodnoty Dataframu)

powers_of_two["Square root"] = np.sqrt(powers_of_two["Power of 2"])
powers_of_two["Log10"] = np.log10(powers_of_two["Power of 2"])
powers_of_two

In [ ]:
# na sloupcích můžeme provádět výpočty i vzájemně

powers_of_two["Sum of all"] = powers_of_two.sum(axis=1)
powers_of_two["N-th root"] = np.power(powers_of_two["Power of 2"][1:],1/powers_of_two.index.values[1:])
powers_of_two

<h5>Měnit lze i jednotlivé hodnoty dataframu

In [ ]:
# změna hodnoty pomocí metody loc()

df_sporitelna.loc[11,"Název protiuctu"]

In [ ]:
df_sporitelna.loc[11,"Název protiuctu"] = "VCELA ZDENEK"
df_sporitelna.loc[11,"Název protiuctu"]

In [ ]:
# podobným způsobem jako u sloupců s nimi počítat a přidávat nové hodnoty

powers_of_two.loc[10, "Power of 2"] = powers_of_two.loc[9, "Power of 2"]*2
powers_of_two

<h5>Výpočet základních statistik částí dataframu</h5>

In [ ]:
# metoda vypisující základní statistiky všech numerických sloupců ve zdrojovém dataframu formou nového dataframu

df_stats = df_csob.describe()
df_stats

# samozřejmě nedává smysl počítat statistiky pro čisla účtů či konstantní symboly, pandas je pouze vidí jako numerické hodnoty

In [ ]:
# výpočet můžeme provádět i po jednotlivých sloupcích pomocí vestavěných metod modulu pandas

# průměr
df_csob["amount"].mean()

In [ ]:
# směrodatná odchylka
df_csob["deposit"].std()

In [ ]:
# kvantil 0.9
df_sporitelna["Castka"].quantile(q=0.9)

In [ ]:
# produkt (součin) všech hodnot
df_sporitelna["Castka"].prod()

In [ ]:
# minimum, maximum
df_csob["deposit"].max(), df_csob["deposit"].min()

Více o konkrétních statistikách dostupých v pandas na: <https://pandas.pydata.org/docs/user_guide/basics.html#basics-stats>

In [ ]:
# na pandas dataframu lze použít i statistiky z modulu Numpy

# rozsah hodnot (max - min)
np.ptp(df_csob["deposit"])

In [ ]:
# percentil 50
np.percentile(df_csob["amount"],q=0.5)

Více o statistikách v modulu Numpy: <https://numpy.org/doc/stable/reference/routines.statistics.html>

<h5>Pro účely rychlé grafické analýzy spolupracuje modul Pandas s knihovnou matplotlib, která umožňuje grafování hodnot pomocí metody plot</h5>

In [ ]:
# základní čárový graf
df_csob.plot(y=["amount","deposit"])

In [ ]:
# grafu lze jednoduše měnit názvy os apod.
df_csob.plot(y="deposit", use_index=True, kind="line",figsize=(8,5), title="Deposit over time", xlabel="Date",ylabel="Deposit in CZK")

In [ ]:
# bar plot
df_csob.plot(y="amount", xticks=[], use_index=True, kind="bar",figsize=(8,5), title="Deposit over time", xlabel="Date",ylabel="Deposit in CZK")

In [ ]:
# další typy grafů z dataframu powers_of_two
# scatter plot
powers_of_two.plot(y="Power of 2", x="Square root", kind="scatter")

In [ ]:
# další typy grafů z dataframu powers_of_two
# horizontal bar plot
powers_of_two.plot(y="Power of 2", kind="barh")

<h5>Cvičení 2: vytvořte program, který spočítá statistiky smysluplných hodnot z dataframů (pohyby, zůstatek) a vhodně je porovná/vizualizuje v grafech (např. jak vypadá medián pohybů na účtu ČSOB vs. na účtu Č. Spořitelny ve sloupcovém grafu)</h5>

Tentokrát k vizualiaci využijte knihovnu matplotlib. Hodnoty ČSOB vyznačte modře, Spořitelnu červeně a z každé statistiky vytvořte samostatný graf

Potřebná dokumentace:
<br>

<https://matplotlib.org/stable/gallery/index>
<br>

<https://matplotlib.org/stable/gallery/lines_bars_and_markers/bar_colors.html#sphx-glr-gallery-lines-bars-and-markers-bar-colors-py>
<br>

Statistické funkce z předchozích buněk a příkladů

In [85]:
# místo pro váš kód

In [ ]:
# vzorové řešení (spusťte pro kontrolu)

def compute_stats(df,stats):
    stat_dict = {}
    for stat in stats:
        stat_dict[stat[0]] = {
        "Mean":np.mean(df[stat[1]]),
        "Median": np.median(df[stat[1]]),
        "Min": np.min(df[stat[1]]),
        "Max": np.max(df[stat[1]]),
           
    }
    return stat_dict

def plot_stats(dfs, keys):
    bar_colors = ["blue","red"]
    fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(16,9))
    dataframes = list(dfs.keys())
    for stat in keys:
        i = keys.index(stat)
        stat_values = []
        for df in dataframes:
            try:
                stat_values.append(list(dfs[df]["stat_vals"][stat].items()))
            except:
                stat_values.append([(0,0) for _ in range(4)])
        
        for j in range(4): # počet jednotlivých statistik které počítáme
            ax[i][j].bar(dataframes, [val[j][1] for val in stat_values], color=bar_colors,width=0.5)
            ax[i][j].set_title(f"{stat} - {stat_values[0][j][0]}")
            
    plt.subplots_adjust(wspace=0.4)

    plt.show()
    
def main():
    keys = ["Movement","Deposit"]
    dataframes = {
        "CSOB": {
            "val":df_csob,
            "stats":[("Deposit","deposit"), ("Movement","amount")],
            "stat_vals":{}
        },
        "Sporitelna": {
            "val": df_sporitelna,
            "stats": [("Movement", "Castka")],
            "stat_vals": {}
        }
    }
    for df in dataframes.keys():
        dataframes[df]["stat_vals"] = compute_stats(
            dataframes[df]["val"],
            dataframes[df]["stats"],
        )
    plot_stats(dataframes,keys)
    

if __name__ == "__main__":
    main()

<h5>Pokuste se o interpretaci dat a jejich grafů</h5>

Např. průměrné velikosti pohybů nám říkají, že na účtu Spořitelna jsou celkem vyrovnané, zatímco účet ČSOB má výrazně vyšší výdaje